# Python MQTT client for controlling the OpenDrop remotely

This notebook demonstrates a `MQTTProxy` class that is designed to control an OpenDrop exposed over an MQTT-Serial Bridge. It is setup to use the freely available `test.mosquitto.org` broker, but can easily be configured to use a local or cloud-based broker.

## Setup

Before running this notebook, make sure that you have the `OpenDrop MQTT-Serial Bridge` notebook running in another tab.

In [1]:
import paho.mqtt.client as mqtt
# Use json_tricks instead of standard json module to handle serialization
# of numpy arrays
import json_tricks as jt
import numpy as np


def on_response_recieved(client, userdata, message):
    '''
    Callback function to handle mqtt messages. Note that this being called
    from the paho client event loop. Not sure how this is implemented
    (i.e., is it happening in a background thread?). There are likely
    thread safetey implications.
    '''
    payload_str = message.payload.decode("utf-8")
    #print("received message: " , str(payload_str))
    payload = jt.loads(payload_str)
    proxy._response = payload


class MQTTProxy():
    def __init__(self, broker):
        '''
        Initialize an MQTTProxy object.

        Parameters
        ----------
        broker : string
            Url for the MQTT broker.
        '''
        self._broker = broker
        self._client = mqtt.Client("OpenDrop Remote Proxy")
        self._client.connect(broker)
        
        self._client.loop_start()
        self._client.subscribe("opendrop/response")
        self._client.on_message=on_response_recieved
        self._response = {}
        
    def identify(self):
        self._client.publish("opendrop/command",
                             jt.dumps({'command': 'identify'}))
        return self.wait_for_response()

    def wait_for_response(self):
        # TODO: Fix this super hacky way to wait for a response and use
        # something that is thread safe (e.g., events).
        while self._response == {}:
            pass
        payload = self._response
        self._response = {}
        return payload['return']

    @property
    def voltage(self,):
        self._client.publish("opendrop/command",
                             jt.dumps({'command': 'voltage', 'type': 'property'}))
        return self.wait_for_response()
    
    @voltage.setter
    def voltage(self, voltage):
        self._client.publish("opendrop/command",
                             jt.dumps({'command': 'voltage',
                                         'args': [voltage], 'type': 'property'}))

    @property
    def state_of_channels(self):
        self._client.publish("opendrop/command",
                             jt.dumps({'command': 'state_of_channels', 'type': 'property'}))
        return self.wait_for_response()
    
    def set_state_of_channels(self, value):
        self._client.publish("opendrop/command",
                             jt.dumps({'command': 'set_state_of_channels',
                                       'args': [value]}))

In [2]:
# Create a mqtt proxy object
proxy = MQTTProxy("test.mosquitto.org")

In [3]:
# Get the identity of the instrument
proxy.identify()

'GaudiLabs,OpenDrop,#00,v3.2'

In [4]:
# Set the voltage
proxy.voltage = 200

In [5]:
# Get the current voltage
proxy.voltage

200.0

In [6]:
# Turn on channels 10, 20 and 30
state_of_channels = np.zeros(128)
state_of_channels[[10, 20, 30]] = 1
proxy.set_state_of_channels(state_of_channels)

# Get the current state of the channels
proxy.state_of_channels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [7]:
# Sweep across channels 1-10
for i in range(10):
    state_of_channels = np.zeros(128)
    state_of_channels[i] = 1
    proxy.set_state_of_channels(state_of_channels)
    
# Get the current state of the channels
proxy.state_of_channels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)